# Econometría Aplicada 1 (Taller) - Examen Final
Importar librerías

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

## Ejercicio 3: Índice hedónico de computadoras
Cargar datos

In [42]:
df = pd.read_csv('../dat/chow.txt', sep='\t')

### (a) Construcción de atributos
Crear los atributos necesarios

In [43]:
# Agregar término constante
df['CONST'] = 1

# Crear atributos logarítmicos
for col in ['RENT','MULT','ACCESS','ADD']:
    df['LN'+col] = np.log(df[col])
    
# Crear Memory Space
df['LNMEM'] = np.log(df[['WORDS','BINARY','DIGITS']].product(axis=1))

# Crear dummies por año (61, 62, ..., 65)
df['D'] = np.where(df['YEAR'].lt(61), 60, df['YEAR'])
df = pd.get_dummies(df, prefix='D', prefix_sep='_', columns=['D'], drop_first=True)

Comparación de matríces de correlación en los periodos
- [1954-1959]

In [44]:
c1 = df.loc[df['YEAR'].between(54,59), 'LNRENT':'LNMEM'].corr()
c1.round(2)

,LNRENT,LNMULT,LNACCESS,LNADD,LNMEM
LNRENT,1.00,-0.87,-0.85,-0.84,0.88
LNMULT,-0.87,1.00,0.86,0.98,-0.74
LNACCESS,-0.85,0.86,1.00,0.85,-0.61
LNADD,-0.84,0.98,0.85,1.00,-0.73
LNMEM,0.88,-0.74,-0.61,-0.73,1.00


- [1960-1965]

In [45]:
c2 = df.loc[df['YEAR'].between(60,65), 'LNRENT':'LNMEM'].corr()
c2.round(2)

,LNRENT,LNMULT,LNACCESS,LNADD,LNMEM
LNRENT,1.00,-0.63,-0.58,-0.63,0.88
LNMULT,-0.63,1.00,0.77,0.94,-0.57
LNACCESS,-0.58,0.77,1.00,0.81,-0.46
LNADD,-0.63,0.94,0.81,1.00,-0.56
LNMEM,0.88,-0.57,-0.46,-0.56,1.00


Los coeficientes de correlación entre las siguientes variables cambiaron menos de 10 puntos porcentuales entre ambos periodos:
1. `LNRENT` y `LNMEM`
1. `LNMULT` y `LNACCESS`
1. `LNMULT` y `LNADD`
1. `LNACCESS` y `LNADD`

Los demás pares de variables cambian en 10 o más puntos porcentuales.

En general, ambas matríces de correlación se parecen. Sin embargo, considero que hay diferencias importantes entre ambos periodos. En particular, hay menos pares de variables con coeficientes de correlación mayores a 0.7 en valor absoluto en el segundo periodo.

Igualmente, creo que Chow estaba en lo correcto al preocuparse por la posibilidad de un problema de multicolinealidad, pues todos los pares de variables excepto por `LNMEM` y `LNACCESS` tienen un coeficiente de correlación de almenos 0.7 en valor absoluto en el periodo previo a 1960.

Las correlaciones siguen siendo altas de 1960 en adelante. Sin embargo, los coeficientes tienden a disminuir en esta ventana de tiempo.

### (b) Regresión de Chow

In [51]:
# Objetos para regresión
feats = ['CONST','D_61','D_62','D_63','D_64','D_65','LNMULT','LNMEM','LNACCESS']
targt = 'LNRENT'
X = df.loc[df['YEAR'].ge(60), feats]
y = df.loc[df['YEAR'].ge(60), targt]

# Modelo de Chow
m1 = sm.OLS(endog=y, exog=X)
m1_res = m1.fit()

In [59]:
print(m1_res.summary().tables[1])
print('Observations:', int(m1_res.nobs))
print('R2:', round(m1_res.rsquared, 3))

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CONST         -0.1045      0.315     -0.332      0.741      -0.732       0.523
D_61          -0.1398      0.166     -0.840      0.404      -0.472       0.192
D_62          -0.4891      0.174     -2.815      0.006      -0.835      -0.143
D_63          -0.5938      0.166     -3.575      0.001      -0.925      -0.263
D_64          -0.9248      0.166     -5.561      0.000      -1.256      -0.593
D_65          -1.1632      0.166     -7.003      0.000      -1.494      -0.832
LNMULT        -0.0654      0.028     -2.301      0.024      -0.122      -0.009
LNMEM          0.5793      0.035     16.369      0.000       0.509       0.650
LNACCESS      -0.1406      0.029     -4.794      0.000      -0.199      -0.082
Observations: 82.0
R2: 0.908
